![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Deploy Azure Arc Data Controller on an existing cluster
 
This notebook walks through the process of deploying a <a href="https://azure.microsoft.com/services/azure-arc/">Azure Arc controller</a> on an existing cluster.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* The **Required information** will check and prompt you for password if it is not set in the environment variable. The password can be used to access the data controller.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Prerequisites** 
Ensure the following tools are installed and added to PATH before proceeding.
 
|Tools|Description|Installation|
|---|---|---|
|kubectl | Command-line tool for monitoring the underlying Kubernetes cluster | [Installation](https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management) |
|azdata | Command-line tool for installing and managing resources in an Azure Arc cluster |[Installation](https://github.com/microsoft/Azure-data-services-on-Azure-Arc/blob/master/scenarios/001-install-client-tools.md) |

### **Setup**

In [ ]:
import pandas,sys,os,json,html,getpass,time, tempfile
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')
def run_command(command):
    print("Executing: " + command)
    !{command}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')

### **Set variables**
Generated by Azure Data Studio using the values collected in the Azure Arc Data controller create wizard

### **Check dependencies**

In [ ]:
run_command('azdata --version')

### **Required information**

In [ ]:
if "AZDATA_NB_VAR_ARC_DOCKER_PASSWORD" in os.environ:
    arc_docker_password = os.environ["AZDATA_NB_VAR_ARC_DOCKER_PASSWORD"]
if "AZDATA_NB_VAR_ARC_ADMIN_PASSWORD" in os.environ:
    arc_admin_password = os.environ["AZDATA_NB_VAR_ARC_ADMIN_PASSWORD"]
else:
    if arc_admin_password == "":
        arc_admin_password = getpass.getpass(prompt = 'Azure Arc Data controller password')
        if arc_admin_password == "":
            sys.exit(f'Password is required.')
        confirm_password = getpass.getpass(prompt = 'Confirm password')
        if arc_admin_password != confirm_password:
            sys.exit(f'Passwords do not match.')

### **Set and show current context**

In [ ]:
os.environ["KUBECONFIG"] = arc_config_file
run_command(f'kubectl config use-context {arc_cluster_context}')
run_command('kubectl config current-context')

### **Create Azure Arc Data controller**

In [ ]:
print (f'Creating Azure Arc controller: {arc_data_controller_name} using configuration {arc_cluster_context}')
os.environ["ACCEPT_EULA"] = 'yes'
os.environ["AZDATA_USERNAME"] = arc_admin_username
os.environ["AZDATA_PASSWORD"] = arc_admin_password
os.environ["DOCKER_USERNAME"] = arc_docker_username
os.environ["DOCKER_PASSWORD"] = arc_docker_password
if os.name == 'nt':
    print(f'If you don\'t see output produced by azdata, you can run the following command in a terminal window to check the deployment status:\n\t {os.environ["AZDATA_NB_VAR_KUBECTL"]} get pods -A')
run_command(f'azdata arc dc create --connectivity-mode {arc_data_controller_connectivity_mode} -n {arc_data_controller_name} -ns {arc_data_controller_namespace} -s {arc_subscription} -g {arc_resource_group} -l {arc_data_controller_location} -p {arc_profile}')
print(f'Azure Arc Data controller cluster: {arc_data_controller_name} created.') 

### **Setting context to created Azure Arc Data controller**

In [ ]:
# Setting context to data controller.
#
run_command(f'kubectl config set-context --current --namespace {arc_data_controller_namespace}')

### **Login to the data controller.**


In [ ]:
# Login to the data controller.
#
run_command(f'azdata login -n {arc_data_controller_namespace}')